In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os

2024-10-30 12:09:45.459710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730270385.475577    4063 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730270385.480274    4063 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 12:09:45.496506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# **1. Data Loading Using ImageDataGenerator**

def load_image_data(img_dir, img_size=(128, 128), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        img_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        img_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False  # Important to keep order for correct labels
    )

    return train_generator, validation_generator

In [3]:
# **2. CNN Model Architecture**

def cnn_classifier(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# **3. Train the CNN Model**

def train_cnn(train_generator, validation_generator, input_shape, num_classes, epochs=10):
    model = cnn_classifier(input_shape, num_classes)

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )

    model.save("cnn_model_images.h5")
    return model

In [5]:
# **4. Calculate Confusion Matrix**

def calculate_confusion_matrix(model, validation_generator):
    # Get true labels
    true_labels = validation_generator.classes
    class_labels = list(validation_generator.class_indices.keys())

    # Get predictions
    predictions = model.predict(validation_generator)
    predicted_labels = np.argmax(predictions, axis=1)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    print("Confusion Matrix:")
    print(conf_matrix)
    print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels, target_names=class_labels))

In [6]:
# **5. Main Program Execution**

if __name__ == "__main__":
    img_dir = "./dataset"
    img_size = (128, 128)
    batch_size = 32

    train_generator, validation_generator = load_image_data(img_dir, img_size=img_size, batch_size=batch_size)
    input_shape = (img_size[0], img_size[1], 3)
    num_classes = len(train_generator.class_indices)

    trained_model = train_cnn(train_generator, validation_generator, input_shape, num_classes, epochs=10)

    # Evaluate the model
    test_loss, test_acc = trained_model.evaluate(validation_generator)
    print(f"Test accuracy: {test_acc * 100:.2f}%")

    # Calculate confusion matrix
    calculate_confusion_matrix(trained_model, validation_generator)

Found 4796 images belonging to 2 classes.
Found 1198 images belonging to 2 classes.


/home/harsh/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1730270448.957830    4063 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4080 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/harsh/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1730270451.991740    4178 service.cc:148] XLA service 0x7ff1ac005e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730270451.991788    4178 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-10-30 12:10:52.043257: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730270452.207721    4178 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-10-30 12:10:52.617412: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,128,128]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"

  2/150 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6562 - loss: 1.5578

...

2024-10-30 12:11:06.734507: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[28,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,3,128,128]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-10-30 12:11:06.775750: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[28,64,61,61]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,32,63,63]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.5946 - loss: 0.7756

2024-10-30 12:11:48.915015: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,128,128]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-10-30 12:11:48.957207: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,61,61]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,63,63]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

150/150 ━━━━━━━━━━━━━━━━━━━━ 72s 454ms/step - accuracy: 0.5950 - loss: 0.7746 - val_accuracy: 0.6469 - val_loss: 0.6077
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 213ms/step - accuracy: 0.7937 - loss: 0.4213 - val_accuracy: 0.8155 - val_loss: 0.4063
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 31s 208ms/step - accuracy: 0.8769 - loss: 0.2862 - val_accuracy: 0.7838 - val_loss: 0.4912
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 204ms/step - accuracy: 0.9086 - loss: 0.2198 - val_accuracy: 0.9282 - val_loss: 0.2181
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 216ms/step - accuracy: 0.9379 - loss: 0.1563 - val_accuracy: 0.8548 - val_loss: 0.4724
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 214ms/step - accuracy: 0.9622 - loss: 0.1022 - val_accuracy: 0.9098 - val_loss: 0.3247
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 218ms/step - accuracy: 0.9549 - loss: 0.1018 - val_accuracy: 0.8280 - val_loss: 0.4559
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 41s 274ms/step - accuracy: 0.9454 - loss: 0.1050 - val

38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 166ms/step - accuracy: 0.8896 - loss: 0.4133
Test accuracy: 91.40%
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 163ms/step
Confusion Matrix:
[[560  62]
 [ 53 523]]

Classification Report:
              precision    recall  f1-score   support

 Handwritten       0.91      0.90      0.91       622
     Printed       0.89      0.91      0.90       576

    accuracy                           0.90      1198
   macro avg       0.90      0.90      0.90      1198
weighted avg       0.90      0.90      0.90      1198

